In [1]:
import numpy as np
from itertools import combinations, product
import random

In [2]:
tol = 1e-10  # tolerance is necessary, det may be ~1e-16

def classify_pts(pts, coeffs, rhs):
    """
    Plug in a set of points to a hyperplane and classify whether
    they lie on the hyperplane or which side of the hyperplane
    """
    less_than = set()
    on_plane = set()
    grtr_than = set()
    for pt in pts:
        res = np.matmul(coeffs, pt)
        if res < rhs - tol:
            less_than.add(pt)
        elif res > rhs + tol:
            grtr_than.add(pt)
        else:
            on_plane.add(pt)
    return less_than, on_plane, grtr_than

def update_mat(mat, i):
    """
    Adds a unit vector to every row of the matrix,
    effectively translating every point(row) a unit distance in the same direction
    """
    n = mat.shape[0]
    unit_vector = np.zeros(n)
    unit_vector[i] = 1
    res = np.empty((0, n))
    for row in mat:
        res = np.vstack((res, row + unit_vector))
    return res

def find_rhs(pt_set, coeffs):
    """
    Find the rhs of a hyperplane, by plugging in pts that lie on that hyperplane
    """
    rhs = np.matmul(coeffs, pt_set[0])
    for pt in pt_set[1:]:
        res = np.matmul(coeffs, pt)
        if abs(res - rhs) > tol:
            print(f"Error: multiple rhs values for {pt_set} with coeffs {coeffs}")
    return rhs  # type scalar

def find_coeffs(pt_set):
    """
    Input pt_set defining a hyperplane, 
    We are able to find a contraint for the hyperplane if the det(matrix) != 0
    Translate points if necessary to find nonzero det.
    If points are lin dep, return None
    """
    n = len(pt_set[0])
    mat = np.array(pt_set)
    coeffs = None
    if abs(np.linalg.det(mat)) > tol:
        coeffs = np.matmul(np.linalg.inv(mat), np.ones((n)))
    else:
        for i in range(n):
            upd_mat = update_mat(mat, i)
            if abs(np.linalg.det(upd_mat)) > tol:
                coeffs = np.matmul(np.linalg.inv(upd_mat), np.ones((n)))
                break
        else:
            pass
            # print(f"Error: coeffs not found for {pt_set}")
    return coeffs

def find_convex_hull(pts):
    """
    Find all supporting hyperplanes of a set of points.
    """
    n = len(pts[0])
    facets = []
    non_facets = []
    for pt_set in combinations(pts, n):
        coeffs = find_coeffs(pt_set)
        if coeffs is None:
            continue
        rhs = find_rhs(pt_set, coeffs)
        (less_than, on_plane, grtr_than) = classify_pts(pts, coeffs, rhs)
        existing_facet = False
        for (_, _, _, facet_set) in facets + non_facets:
            if on_plane == facet_set:
                existing_facet = True
                break
        if existing_facet:
            continue
        if len(less_than) > 0 and len(grtr_than) > 0:
            non_facets.append((coeffs, "==", rhs, on_plane))
            continue
        if len(less_than) == 0 and len(grtr_than) > 0:
            facets.append((coeffs, ">=", rhs, on_plane))
            continue
        if len(less_than) > 0 and len(grtr_than) == 0:
            facets.append((coeffs, "<=", rhs, on_plane))
            continue
        if len(less_than) == 0 and len(grtr_than) == 0:
            facets.append((coeffs, "==", rhs, on_plane))
            continue
    return facets, non_facets

def list_constraints(facets, names=None):
    """
    Use given variable names to pretty print the equations of hyperplanes
    """
    for facet in facets:
        coeffs, sense, rhs, pt_set = facet
        n = len(coeffs)
        if names is None:
            names = [ 'x' + str(i) for i in range(n)]
        print(f"CONSTRAINT for {pt_set}:")
        for idx, x in np.ndenumerate(coeffs):
            if x != 0:
                print(f" + {x} * {names[idx[0]]} ", end='')
        print(f"{sense} {rhs}\n")    

def convex_hull_constraints(pts, names=None):
    """
    Find and print convex hull of a set of points
    """
    facets, _ = find_convex_hull(pts)
    print(f'pts: {pts} \n')
    list_constraints(facets, names)

In [3]:
def rnd_pts_on_hcube(dim, pct):
    """
    Return list of random points on hypercube of dimension dim,
    Return approx pct % of the points
    To return all points, use pct >= 1.0
    """
    hcube = []
    for v in product(range(2), repeat=dim):
        if random.random() < pct:
            hcube.append(v)
    return hcube

In [4]:
unit_sqr = rnd_pts_on_hcube(2, 1.0)
convex_hull_constraints(unit_sqr, names=['x', 'y'])

pts: [(0, 0), (0, 1), (1, 0), (1, 1)] 

CONSTRAINT for {(0, 1), (0, 0)}:
 + 1.0 * x >= 0.0

CONSTRAINT for {(1, 0), (0, 0)}:
 + 1.0 * y >= 0.0

CONSTRAINT for {(0, 1), (1, 1)}:
 + 1.0 * y <= 1.0

CONSTRAINT for {(1, 0), (1, 1)}:
 + 1.0 * x <= 1.0



In [5]:
cube = rnd_pts_on_hcube(3, 1.0)
convex_hull_constraints(cube)

pts: [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)] 

CONSTRAINT for {(0, 1, 1), (0, 1, 0), (0, 0, 0), (0, 0, 1)}:
 + 1.0 * x0 >= 0.0

CONSTRAINT for {(1, 0, 0), (0, 0, 0), (0, 0, 1), (1, 0, 1)}:
 + 1.0 * x1 >= 0.0

CONSTRAINT for {(1, 0, 0), (1, 1, 0), (0, 1, 0), (0, 0, 0)}:
 + 1.0 * x2 >= 0.0

CONSTRAINT for {(0, 1, 1), (1, 1, 1), (0, 0, 1), (1, 0, 1)}:
 + 1.0 * x2 <= 1.0

CONSTRAINT for {(0, 1, 1), (0, 1, 0), (1, 1, 0), (1, 1, 1)}:
 + 1.0 * x1 <= 1.0

CONSTRAINT for {(1, 0, 0), (1, 1, 0), (1, 1, 1), (1, 0, 1)}:
 + 1.0 * x0 <= 1.0



In [6]:
rcube = rnd_pts_on_hcube(3, 0.5)
convex_hull_constraints(rcube)

pts: [(0, 0, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)] 

CONSTRAINT for {(0, 1, 1), (0, 0, 0), (1, 0, 1)}:
 + 1.0 * x0  + 1.0 * x1  + -1.0 * x2 >= 0.0

CONSTRAINT for {(0, 1, 1), (0, 0, 0), (1, 1, 0)}:
 + 1.0 * x0  + -1.0 * x1  + 1.0 * x2 >= 0.0

CONSTRAINT for {(1, 0, 0), (0, 0, 0), (1, 0, 1)}:
 + 1.0 * x1 >= 0.0

CONSTRAINT for {(1, 0, 0), (1, 1, 0), (0, 0, 0)}:
 + 1.0 * x2 >= 0.0

CONSTRAINT for {(0, 1, 1), (1, 1, 1), (1, 0, 1)}:
 + 1.0 * x2 <= 1.0

CONSTRAINT for {(0, 1, 1), (1, 1, 1), (1, 1, 0)}:
 + 1.0 * x1 <= 1.0

CONSTRAINT for {(1, 0, 0), (1, 1, 0), (1, 1, 1), (1, 0, 1)}:
 + 1.0 * x0 <= 1.0



In [7]:
pts = [
(0, 0, 0),
(1, 1, 0),
(0, 1, 1),
(1, 0, 1),
(1, 1, 1)]
n = len(pts[0])
m = len(pts)
print(f'{m} pts of dimension {n}')
varnames = ['x', 'y', 'z']
convex_hull_constraints(pts, varnames)

5 pts of dimension 3
pts: [(0, 0, 0), (1, 1, 0), (0, 1, 1), (1, 0, 1), (1, 1, 1)] 

CONSTRAINT for {(1, 1, 0), (0, 1, 1), (0, 0, 0)}:
 + 1.0 * x  + -1.0 * y  + 1.0 * z >= 0.0

CONSTRAINT for {(1, 1, 0), (0, 0, 0), (1, 0, 1)}:
 + 1.0 * x  + -1.0 * y  + -1.0 * z <= 0.0

CONSTRAINT for {(0, 1, 1), (0, 0, 0), (1, 0, 1)}:
 + 1.0 * x  + 1.0 * y  + -1.0 * z >= 0.0

CONSTRAINT for {(1, 1, 0), (0, 1, 1), (1, 1, 1)}:
 + 1.0 * y <= 1.0

CONSTRAINT for {(1, 1, 0), (1, 1, 1), (1, 0, 1)}:
 + 1.0 * x <= 1.0

CONSTRAINT for {(0, 1, 1), (1, 1, 1), (1, 0, 1)}:
 + 1.0 * z <= 1.0



In [8]:
pts1 = [
(0, 0, 0, 0),
(0, 0, 1, 1),
(0, 0, 1, 0),
(0, 1, 0, 0),
(1, 0, 0, 1),
(1, 1, 0, 0)]
names1 = ['ramp_prev', 'ramp', 'fo_prev', 'fo']
convex_hull_constraints(pts1, names1)

pts: [(0, 0, 0, 0), (0, 0, 1, 1), (0, 0, 1, 0), (0, 1, 0, 0), (1, 0, 0, 1), (1, 1, 0, 0)] 

CONSTRAINT for {(0, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 1)}:
 + 1.0 * ramp_prev >= 0.0

CONSTRAINT for {(0, 0, 0, 0), (1, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1)}:
 + 1.0 * ramp >= 0.0

CONSTRAINT for {(0, 0, 0, 0), (1, 0, 0, 1), (0, 1, 0, 0), (0, 0, 1, 1)}:
 + 1.0 * ramp_prev  + 1.0 * fo_prev  + -1.0 * fo >= 0.0

CONSTRAINT for {(1, 1, 0, 0), (0, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0)}:
 + 1.0 * fo >= 0.0

CONSTRAINT for {(1, 1, 0, 0), (0, 0, 0, 0), (1, 0, 0, 1), (0, 0, 1, 0)}:
 + 1.0 * ramp_prev  + -1.0 * ramp  + -1.0 * fo <= 0.0

CONSTRAINT for {(1, 1, 0, 0), (0, 0, 0, 0), (1, 0, 0, 1), (0, 1, 0, 0)}:
 + 1.0 * fo_prev >= 0.0

CONSTRAINT for {(1, 1, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 1)}:
 + 1.0 * ramp  + 1.0 * fo_prev <= 1.0

CONSTRAINT for {(1, 1, 0, 0), (1, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1)}:
 + 1.0 * ramp_prev  + 1.0 * fo_prev <= 1.0

CONSTRAINT for {(1, 1, 0, 0), (1, 0

In [9]:
hcube4 = rnd_pts_on_hcube(4, 1.0)
names4=['x', 'y', 'z', 'w']
convex_hull_constraints(hcube4, names=names4)

pts: [(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1), (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1), (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1)] 

CONSTRAINT for {(0, 0, 0, 1), (0, 1, 1, 1), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 0), (0, 1, 1, 0), (0, 0, 1, 1), (0, 1, 0, 1)}:
 + 1.0 * x >= 0.0

CONSTRAINT for {(1, 0, 1, 1), (1, 0, 0, 0), (1, 0, 1, 0), (0, 0, 0, 1), (1, 0, 0, 1), (0, 0, 1, 0), (0, 0, 0, 0), (0, 0, 1, 1)}:
 + 1.0 * y >= 0.0

CONSTRAINT for {(1, 1, 0, 1), (1, 0, 0, 0), (1, 1, 0, 0), (0, 0, 0, 1), (1, 0, 0, 1), (0, 1, 0, 0), (0, 0, 0, 0), (0, 1, 0, 1)}:
 + 1.0 * z >= 0.0

CONSTRAINT for {(1, 0, 0, 0), (1, 1, 1, 0), (1, 0, 1, 0), (1, 1, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 0), (0, 1, 1, 0)}:
 + 1.0 * w >= 0.0

CONSTRAINT for {(1, 0, 1, 1), (1, 1, 0, 1), (0, 0, 0, 1), (0, 1, 1, 1), (1, 0, 0, 1), (1, 1, 1, 1), (0, 1, 0, 1), (0, 0, 1, 1)}:
 + 1.0 * w <= 1.0

CONSTRAINT for {(1, 0, 1, 1)

In [10]:
facets, non_facets = find_convex_hull(hcube4)
list_constraints(non_facets, names4)

CONSTRAINT for {(1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 0, 0), (0, 0, 0, 1), (1, 1, 1, 1), (0, 0, 1, 0), (0, 0, 0, 0), (0, 0, 1, 1)}:
 + 1.0 * x  + -1.0 * y == 0.0

CONSTRAINT for {(1, 0, 1, 1), (1, 1, 1, 0), (1, 0, 1, 0), (0, 0, 0, 1), (1, 1, 1, 1), (0, 1, 0, 0), (0, 0, 0, 0), (0, 1, 0, 1)}:
 + 1.0 * x  + -1.0 * z == 0.0

CONSTRAINT for {(1, 0, 0, 0), (1, 1, 1, 0), (0, 0, 0, 1), (0, 1, 1, 1), (1, 0, 0, 1), (1, 1, 1, 1), (0, 0, 0, 0), (0, 1, 1, 0)}:
 + 1.0 * y  + -1.0 * z == 0.0

CONSTRAINT for {(1, 0, 1, 1), (1, 0, 1, 0), (0, 0, 0, 1), (0, 1, 1, 1), (0, 0, 0, 0), (0, 1, 1, 0)}:
 + 1.0 * x  + 1.0 * y  + -1.0 * z == 0.0

CONSTRAINT for {(1, 1, 0, 1), (1, 1, 0, 0), (0, 0, 0, 1), (0, 1, 1, 1), (0, 0, 0, 0), (0, 1, 1, 0)}:
 + 1.0 * x  + -1.0 * y  + 1.0 * z == 0.0

CONSTRAINT for {(1, 0, 1, 1), (1, 1, 0, 1), (1, 0, 1, 0), (0, 0, 0, 1), (1, 1, 0, 0), (0, 0, 0, 0)}:
 + 1.0 * x  + -1.0 * y  + -1.0 * z == 0.0

CONSTRAINT for {(1, 0, 1, 1), (1, 1, 0, 1), (1, 0, 0, 1), (0, 1, 0, 0), (0, 0, 1, 0), (1, 

In [11]:
hcube4r = rnd_pts_on_hcube(4, 0.5)
convex_hull_constraints(hcube4r, names4)

pts: [(0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 1), (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 1, 0, 1), (1, 1, 1, 0)] 

CONSTRAINT for {(0, 1, 1, 1), (0, 1, 0, 0), (0, 0, 1, 0), (0, 1, 0, 1), (0, 0, 1, 1)}:
 + 1.0 * x >= 0.0

CONSTRAINT for {(1, 0, 0, 0), (1, 0, 0, 1), (0, 1, 0, 0), (0, 0, 1, 0), (0, 1, 0, 1), (0, 0, 1, 1)}:
 + 1.0 * x  + 1.0 * y  + 1.0 * z >= 1.0

CONSTRAINT for {(1, 1, 1, 0), (1, 0, 1, 0), (0, 1, 1, 1), (0, 0, 1, 0), (0, 0, 1, 1)}:
 + 1.0 * z <= 1.0

CONSTRAINT for {(1, 0, 0, 0), (1, 0, 1, 0), (1, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1)}:
 + 1.0 * y >= 0.0

CONSTRAINT for {(0, 1, 1, 1), (1, 1, 1, 0), (0, 1, 0, 0), (0, 0, 1, 0)}:
 + -1.0 * x  + 1.0 * y  + 1.0 * z  + -1.0 * w <= 1.0

CONSTRAINT for {(1, 1, 1, 0), (1, 0, 0, 0), (1, 0, 1, 0), (0, 1, 0, 0), (0, 0, 1, 0)}:
 + 1.0 * w >= 0.0

CONSTRAINT for {(1, 1, 0, 1), (0, 1, 1, 1), (1, 0, 0, 1), (0, 1, 0, 1), (0, 0, 1, 1)}:
 + 1.0 * w <= 1.0

CONSTRAINT for {(1, 1, 0, 1), (1, 1, 1, 0), (1, 0, 1,